In [6]:
import json
import openai
from dotenv import load_dotenv
from tqdm import tqdm
import os

# ✅ .env 파일에서 API 키 불러오기
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# ✅ 상위 타입 설명 + 하위 타입 예시
system_prompt = """
You are a deceptive pattern classifier. You must classify each input pattern into ONE of the following 7 categories:

1. social_proof
- Definition: Tricks the user into believing that many people are interested in a product or have recently purchased it.
- Examples: fake customer activity popups, fake testimonials, "X people are viewing this item now"

2. misdirection
- Definition: Confuses users by altering expected interface behavior, such as placing buttons in unusual locations, using misleading hierarchy, or emphasizing the wrong action.
- Examples: confirmshaming, trick wording, visually obscured alternatives

3. urgency
- Definition: Creates pressure by claiming time is running out to push users into making a quick decision.
- Examples: countdown timers, "sale ends in 5 minutes", "act now"

4. forced_action
- Definition: Requires users to complete an unrelated action in order to proceed, such as signing up before viewing information.
- Examples: forced login to see details, mandatory email access, bundled consent

5. obstruction
- Definition: Makes it difficult for users to cancel or opt out, usually requiring more steps than sign-up.
- Examples: hidden cancellation buttons, requiring customer service calls, unclear opt-out

6. sneaking
- Definition: Hides or adds unexpected charges or items during checkout without the user's awareness.
- Examples: hidden fees, hidden subscriptions, pre-selected upsells

7. scarcity
- Definition: Falsely claims a product is in limited supply to trigger fear of missing out.
- Examples: "only 2 left", fake stock counters, fake demand messages

Classify each pattern below into one of the above categories. Respond only with the category name, like: "sneaking" or "misdirection".
"""

# ✅ 사용자 입력용 프롬프트 생성 함수
def create_user_prompt(name, definition, example):
    return f"""
Pattern Name: {name}
Definition: {definition}
Example: {example}

Which category does this pattern belong to?
"""

# ✅ JSON 파일 읽기
with open("dark_patterns.json", "r", encoding="utf-8") as f:
    dark_patterns = json.load(f)

# ✅ 분류 결과 저장용 dict
results = {}

# ✅ 하나씩 처리
for name, item in tqdm(dark_patterns.items()):
    try:
        user_prompt = create_user_prompt(name, item["definition"], item["example"])

        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0
        )

        category = response["choices"][0]["message"]["content"].strip().lower()
        results[name] = category
        print(f"[✓] {name} → {category}")

    except Exception as e:
        print(f"[X] {name} 처리 중 오류: {e}")
        results[name] = "error"

# ✅ 결과 저장
with open("dark_patterns_mapped.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("\n🎉 분류 완료! 결과는 'dark_patterns_mapped.json'에 저장됨.")


100%|██████████| 16/16 [00:00<00:00, 13187.04it/s]

[X] comparison_prevention 처리 중 오류: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.
[X] confirmshaming 처리 중 오류: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.
[X] Disguised_ads 처리 중 오류: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_A